## Neural network implement of Logistic regression
In the previous sections, we use gradient descent to run the Logistic regression model. We mentioned some important concepts, like epochs, mini-batch, etc.. We are going to use `PyTorch` to implement it. We will reuse many codes we wrote in the previous chapter.
<!-- 
### A simple example

We 

$$f(x)$$ -->



### Example

We still use the horse colic dataset as an example. We first prepare the dataset.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/horse-colic/horse-colic.data'
df = pd.read_csv(url, sep='\\s+', header=None)
df = df.replace("?", np.NaN)

df.fillna(0, inplace=True)
df = df.drop(columns=[2, 24, 25, 26, 27])
df[23] = df[23].replace({1: 1, 2: 0})
X = df.iloc[:, :-1].to_numpy().astype(float)
y = df[23].to_numpy().astype(int)

SEED = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=SEED)

We need to perform normalization before throwing the data into the model. Here we use the `MinMaxScaler()` from `sklearn` package. 


In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train = mms.fit_transform(X_train, y_train)
X_test = mms.transform(X_test)

Then we write a `Dataset` class to build the dataset and create the dataloaders. Since the set is already split, we don't need to `random_split` here.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyData(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=float)
        self.y = torch.tensor(y, dtype=float).reshape(-1, 1)

    def __getitem__(self, index):
        return (self.X[index], self.y[index])

    def __len__(self):
        return len(self.y)


train_set = MyData(X_train, y_train)
val_set = MyData(X_test, y_test)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32)

In [ ]:
#| echo: false
#| output: false

from assests.code.models import ModelTemplate

In the following code, we first set up the original model.


In [ ]:
import torch.nn as nn
from torch.nn.modules import Linear

class LoR(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.linear = Linear(in_features=22, out_features=1, dtype=float)
        self.activation = nn.Sigmoid()

    def forward(self, X):
        # pred = self.activation(self.linear(X))
        pred = self.linear(X)
        # return (pred >= 0).float()
        return pred

Then we derive the base `ModelTemplate` class.


In [ ]:
class LoRModel(ModelTemplate):
    def __init__(self, model, loss_fn, optimizer):
        super().__init__(model, loss_fn, optimizer)
        self.stats['acc_train'] = []
        self.stats['acc_val'] = []

    def compute_acc(self, dataloader):
        with torch.no_grad():
            acc = []
            for X_batch, y_batch in dataloader:
                yhat = torch.sigmoid(self.model(X_batch))
                y_pred = (yhat>=0.5).to(float)
                acc.append((y_pred==y_batch).sum().item())
            # print(acc_train)
        return np.sum(acc)/len(dataloader.dataset)

    def log_update(self, train_time, loss, val_time, val_loss, train_loader, val_loader):
        super().log_update(train_time, loss, val_time, val_loss, train_loader, val_loader)
        acc_train = self.compute_acc(train_loader)
        acc_val = self.compute_acc(val_loader)
        self.stats['acc_train'].append(acc_train)
        self.stats['acc_val'].append(acc_val)


        # p = self.model.state_dict()
        # self.stats['acc'].append([p['linear.bias'].item(), p['linear.weight'].item()])

    def log_output(self, verbose=0):
        s = super().log_output(verbose=0, formatstr=':.6f')
        s.append(f'acc_train: {self.stats['acc_train'][-1]:.6f}')
        s.append(f'acc_val: {self.stats['acc_val'][-1]:.6f}')
        # s.append(f'p: [{self.stats['p'][-1][0]:.6f}, {self.stats['p'][-1][1]:.6f}]')
        if verbose == 1:
            print(' '.join(s))
        return s

In [ ]:
from torch.optim import SGD
from torch.nn import BCEWithLogitsLoss, BCELoss

original_model = LoR()
model = LoRModel(model=original_model, loss_fn=BCEWithLogitsLoss(),
                 optimizer=SGD(original_model.parameters(), lr = 0.1))

model.train(train_loader, val_loader, epoch_num=100, verbose=1)